In [7]:
import symforce
symforce.set_epsilon_to_symbol()
# https://github.com/symforce-org/symforce/issues/299
# https://github.com/symforce-org/symforce/issues/300
# https://colab.research.google.com/drive/11G80afyuW_Wxse4o4-Nikb8syi38Mjlk?usp=sharing#scrollTo=Xn3SeYzyRCTL

In [8]:
# -----------------------------------------------------------------------------
# Load the .g2o file
# -----------------------------------------------------------------------------
import numpy as np

from symforce import typing as T
from symforce.values import Values

In [9]:
import symforce.symbolic as sf
from symforce import logger

In [10]:
def info2mat(info):
    mat = np.zeros((6,6))
    ix = 0
    for i in range(mat.shape[0]):
        mat[i,i:] = info[ix:ix+(6-i)]
        mat[i:,i] = info[ix:ix+(6-i)]
        ix += (6-i)
    return mat

In [11]:
# sf.Rot3([1,0,0,0])
# display()
sf.Rot3(sf.Quaternion(xyz= np.asarray([0,0,0]), w = 1))
# sf.Quaternion(xyz=sf.V3(*xyz), w=w)

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

In [12]:
def printData(filename_path: str):
    with open(filename_path) as f:
            for line in f:
                line = line.split()
                # print(line)
                if line[0] == 'VERTEX_SE3:QUAT':
                    # ids.append(sf.Scalar(int(line[1])))
                    # nodes.append(sf.Pose3(R = sf.Rot3(
                    #                 sf.Quaternion(xyz= np.array(line[5:-1], dtype=np.float64), w = float(line[-1]))), 
                    #                 t = sf.Vector3(np.array(line[2:5], dtype=np.float64))))
                    # pass
                    print(line)
                elif line[0] == 'EDGE_SE3:QUAT':
                    # edges.append(sf.Vector2(np.array(line[1:3], dtype=np.int64)))
                    # poses.append(sf.Pose3(R = sf.Rot3(
                    #                 sf.Quaternion(xyz= np.array(line[6:9], dtype=np.float64), w = float(line[9]))), 
                    #                 t = sf.Vector3(np.array(line[3:6], dtype=np.float64))))
                    # infos.append()
                    print(line)
                
# printData('/home/vrex/symForce/symforce_ws/pose_graph_opt/data/parking-garage.g2o')

In [13]:
def build_values(filename_path: str) -> T.Tuple[Values, int]:
    # np.random.seed(42)

    # Create a problem setup and initial guess
    values = Values()

    ids = []
    poses= []
    nodes = []
    edges= []
    infos = []
    with open(filename_path) as f:
        for line in f:
            line = line.split()
            if line[0] == 'VERTEX_SE3:QUAT':
                ids.append(sf.Scalar(int(line[1])))
                nodes.append(sf.Pose3(R = sf.Rot3(
                                sf.Quaternion(xyz= np.array(line[5:-1], dtype=np.float64), w = float(line[-1]))), 
                                t = sf.Vector3(np.array(line[2:5], dtype=np.float64))))
                
            elif line[0] == 'EDGE_SE3:QUAT':
                edges.append(sf.Vector2(np.array(line[1:3], dtype=np.int64)))
                poses.append(sf.Pose3(R = sf.Rot3(
                                sf.Quaternion(xyz= np.array(line[6:9], dtype=np.float64), w = float(line[9]))), 
                                t = sf.Vector3(np.array(line[3:6], dtype=np.float64))))
                infos.append(sf.Matrix66(info2mat(np.array(line[10:], dtype=np.float64))))

    values["ids"] = np.stack(ids)
    values["poses"] = np.stack(poses)
    values["nodes"] = np.stack(nodes)
    values["edges"] = np.stack(edges)
    values["infos"] = np.stack(infos)
    
    values["epsilon"] = sf.numeric_epsilon

    print("num ids",values["ids"])
    print("num poses",values["poses"])
    print("num nodes",values["nodes"])
    print("num edges",values["edges"])
    print("num infos",values["infos"])
    print('infos',infos[0] )
    print('infos',values["infos"][0] )
    return values, values["nodes"].shape[0]


In [14]:
np.random.normal(size=6)

array([ 0.10533258,  0.48482371, -0.14796068,  1.43987001,  0.16910764,
       -0.29568144])

In [15]:
# Testingn whether the dat loads properly or not
_, _ = build_values('/home/vrex/symForce/symforce_ws/pose_graph_opt/data/parking-garage.g2o')
# 
# TODO test more thoroughly 

num ids [0.000e+00 1.000e+00 2.000e+00 ... 1.658e+03 1.659e+03 1.660e+03]
num poses [<Pose3 R=<Rot3 <Q xyzw=[-0.0107791, 0.00867285, -0.00190021, 0.999902]>>, t=(4.15448, -0.0665288, 0.000389663)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00272799, -0.000777724, 0.00363979, 0.999989]>>, t=(4.15971, -0.0912353, 0.0567356)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.00251893, -0.015912, 0.0148755, 0.99976]>>, t=(4.28985, -0.0247738, -0.00423717)>
 ...
 <Pose3 R=<Rot3 <Q xyzw=[0.00229015, 0.0030218, -0.0257697, 0.999661]>>, t=(4.19805, -0.240542, 0.0378044)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00578879, -0.00019708, -0.00536722, 0.999969]>>, t=(4.24184, -0.061526, 0.0197744)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00395105, 0.00187026, 0.0429952, 0.999066]>>, t=(4.21884, -0.184745, 0.00856371)>]
num nodes [<Pose3 R=<Rot3 <Q xyzw=[0.0, 0.0, 0.0, 1.0]>>, t=(0.0, 0.0, 0.0)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.0107791, 0.00867285, -0.00190021, 0.999902]>>, t=(4.15448, -0.0665288, 0.000389663)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.00802114, 0.00792915, 0.00172

In [26]:
def residual(
    pose: sf.Pose3, node1: sf.Pose3, node2: sf.Pose3
) -> sf.Pose3:
    """
    Residual function
    """
    error = pose.inverse() * node1.inverse() * node2 
    
    # return sf.Pose3(error)
    return error 

# dmcgann

# def odometry_residual(
#     world_T_a: sf.Pose3,
#     world_T_b: sf.Pose3,
#     a_T_b: sf.Pose3,
#     diagonal_sigmas: sf.V6,
#     epsilon: sf.Scalar,
# ) -> sf.V6:
#     """
#     Residual on the relative pose between two timesteps of the robot.

#     Args:
#         world_T_a: First pose in the world frame
#         world_T_b: Second pose in the world frame
#         a_T_b: Relative pose measurement between the poses
#         diagonal_sigmas: Diagonal standard deviation of the tangent-space error
#         epsilon: Small number for singularity handling
#     """
#     a_T_b_predicted = world_T_a.inverse() * world_T_b
#     tangent_error = a_T_b_predicted.local_coordinates(a_T_b, epsilon=epsilon)
#     return T.cast(sf.V6, sf.M.diag(diagonal_sigmas.to_flat_list()).inv() * sf.V6(tangent_error))

In [27]:
from symforce.opt.factor import Factor

def build_factors(num_nodes: int) -> T.Iterator[Factor]:
    """
    Build factors for a problem of the given dimensionality.

    TODO: this looks wrong -> figure it out 


    """

    for i in range(num_nodes - 1):
        yield Factor(
            residual=residual,
            keys=[f"poses[{i}]", 
                  f"nodes[edges[{i},0]]", 
                  f"nodes[edges[{i},1]]", 
                #   "epsilon"
                  ],
        )
    

    # for i in range(num_poses - 1):
    #     yield Factor(
    #         residual=odometry_residual,
    #         keys=[
    #             f"poses[{i}]",
    #             f"poses[{i + 1}]",
    #             f"odometry_relative_pose_measurements[{i}]",
    #             "odometry_diagonal_sigmas",
    #             "epsilon",
    #         ],
    #     )

In [28]:
from symforce.opt.optimizer import Optimizer

_values, _num_nodes = build_values('/home/vrex/symForce/symforce_ws/pose_graph_opt/data/parking-garage.g2o')

num ids [0.000e+00 1.000e+00 2.000e+00 ... 1.658e+03 1.659e+03 1.660e+03]
num poses [<Pose3 R=<Rot3 <Q xyzw=[-0.0107791, 0.00867285, -0.00190021, 0.999902]>>, t=(4.15448, -0.0665288, 0.000389663)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00272799, -0.000777724, 0.00363979, 0.999989]>>, t=(4.15971, -0.0912353, 0.0567356)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.00251893, -0.015912, 0.0148755, 0.99976]>>, t=(4.28985, -0.0247738, -0.00423717)>
 ...
 <Pose3 R=<Rot3 <Q xyzw=[0.00229015, 0.0030218, -0.0257697, 0.999661]>>, t=(4.19805, -0.240542, 0.0378044)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00578879, -0.00019708, -0.00536722, 0.999969]>>, t=(4.24184, -0.061526, 0.0197744)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00395105, 0.00187026, 0.0429952, 0.999066]>>, t=(4.21884, -0.184745, 0.00856371)>]
num nodes [<Pose3 R=<Rot3 <Q xyzw=[0.0, 0.0, 0.0, 1.0]>>, t=(0.0, 0.0, 0.0)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.0107791, 0.00867285, -0.00190021, 0.999902]>>, t=(4.15448, -0.0665288, 0.000389663)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.00802114, 0.00792915, 0.00172

In [29]:
for key, value in _values.items_recursive():
    print(f"{key}: {value}")

ids: [0.000e+00 1.000e+00 2.000e+00 ... 1.658e+03 1.659e+03 1.660e+03]
poses: [<Pose3 R=<Rot3 <Q xyzw=[-0.0107791, 0.00867285, -0.00190021, 0.999902]>>, t=(4.15448, -0.0665288, 0.000389663)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00272799, -0.000777724, 0.00363979, 0.999989]>>, t=(4.15971, -0.0912353, 0.0567356)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.00251893, -0.015912, 0.0148755, 0.99976]>>, t=(4.28985, -0.0247738, -0.00423717)>
 ...
 <Pose3 R=<Rot3 <Q xyzw=[0.00229015, 0.0030218, -0.0257697, 0.999661]>>, t=(4.19805, -0.240542, 0.0378044)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00578879, -0.00019708, -0.00536722, 0.999969]>>, t=(4.24184, -0.061526, 0.0197744)>
 <Pose3 R=<Rot3 <Q xyzw=[0.00395105, 0.00187026, 0.0429952, 0.999066]>>, t=(4.21884, -0.184745, 0.00856371)>]
nodes: [<Pose3 R=<Rot3 <Q xyzw=[0.0, 0.0, 0.0, 1.0]>>, t=(0.0, 0.0, 0.0)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.0107791, 0.00867285, -0.00190021, 0.999902]>>, t=(4.15448, -0.0665288, 0.000389663)>
 <Pose3 R=<Rot3 <Q xyzw=[-0.00802114, 0.00792915, 0.00172397, 0.99

In [30]:
# Create factors
factors = build_factors(num_nodes=_num_nodes)

In [42]:
# Select the keys to optimize - the rest will be held constant

# what this?????????? 
# optimized_keys = [f"poses[{i}]" for i in range(num_nodes)]
_optimized_keys = [f"nodes[{i}]" for i in range(_num_nodes)]

In [43]:
# Create the optimizer
optimizer = Optimizer(
    factors=factors,
    optimized_keys=_optimized_keys,
    debug_stats=True,  # Return problem stats for every iteration
    params=Optimizer.Params(verbose=True),  # Customize optimizer behavior
)

AssertionError: Cannot compute a linearization with respect to 0 arguments

In [ ]:
# Solve and return the result
result = optimizer.optimize(_values)

NameError: name 'optimizer' is not defined

In [ ]:
# Print some values
print(f"Num iterations: {len(result.iterations) - 1}")
print(f"Final error: {result.error():.6f}")

for i, pose in enumerate(result.optimized_values["poses"]):
    print(f"Pose {i}: t = {pose.position()}, heading = {pose.rotation().to_tangent()[0]}")

# Plot the result
# TODO(hayk): mypy gives the below error, but a relative import also doesn't work.
# Skipping analyzing "symforce.examples.robot_2d_localization.plotting":
#     found module but no type hints or library stubs
from symforce.examples.robot_2d_localization.plotting import plot_solution

plot_solution(optimizer, result)

